In [10]:
# Cats and Dogs image classification

In [12]:
# Get session and buckets

In [52]:
import sagemaker

print(sagemaker.__version__)

sess = sagemaker.Session()
bucket = sess.default_bucket()
print(bucket)

2.107.0
sagemaker-us-east-1-829135631045


In [55]:
# 1. Using image format
# prefix = 'dogscats-images'

# s3_train_path = 's3://{}/{}/input/train/'.format(bucket, prefix)
# s3_val_path = 's3://{}/{}/input/val/'.format(bucket, prefix)
# s3_train_lst_path = 's3://{}/{}/input/train_lst/'.format(bucket, prefix)
# s3_val_lst_path = 's3://{}/{}/input/val_lst/'.format(bucket, prefix)

# s3_output = 's3://{}/{}/output/'.format(bucket, prefix)

# print(s3_train_path)
# print(s3_val_path)
# print(s3_train_lst_path)
# print(s3_val_lst_path)
# print(s3_output)

In [57]:
# 2 Train using recordIO
prefix = 'dogscats'
s3_train_path = 's3://{}/{}/input/train/'.format(bucket, prefix)
s3_val_path = 's3://{}/{}/input/validation/'.format(bucket, prefix)

s3_output='s3://{}/{}/output/'.format(bucket, prefix)
print(s3_train_path)
print(s3_val_path)

s3://sagemaker-us-east-1-829135631045/dogscats/input/train/
s3://sagemaker-us-east-1-829135631045/dogscats/input/validation/


In [17]:
# Retrieve image and configure training estimator

In [58]:
from sagemaker.image_uris import retrieve

region = sess.boto_session.region_name    
container = retrieve('image-classification', region)
print(container)



811284229777.dkr.ecr.us-east-1.amazonaws.com/image-classification:1


In [59]:
ic = sagemaker.estimator.Estimator(
    container,
    sagemaker.get_execution_role(), 
    instance_count=1, 
    instance_type='ml.p3.2xlarge',
    output_path=s3_output)

In [60]:
ic

In [61]:
# Set hyperparam for training job
ic.set_hyperparameters(
    num_layers=18,               # Train a Resnet-18 model
    use_pretrained_model=0,      # Train from scratch
    num_classes=2,               # Dogs and cats
    num_training_samples=22500,  # Number of training samples
    mini_batch_size=128,
    resize=224,
    epochs=10)                   # Learn the training set 10 times

In [62]:
# # 1 Using image format
# from sagemaker import TrainingInput

# train_data = TrainingInput(
#     s3_train_path, 
#     content_type='application/x-image')

# val_data = TrainingInput(
#     s3_val_path, 
#     content_type='application/x-image')

# train_lst_data = TrainingInput(
#     s3_train_lst_path, 
#     content_type='application/x-image')

# val_lst_data = TrainingInput(
#     s3_val_lst_path, 
#     content_type='application/x-image')

# s3_channels = {'train': train_data, 'validation': val_data, 
#                'train_lst': train_lst_data, 'validation_lst': val_lst_data}

In [65]:
# 2 Using RecordIO
from sagemaker import TrainingInput

train_data = TrainingInput(
    s3_train_path,
    content_type='application/x-recordio')
validation_data = TrainingInput(
    s3_val_path,                                        
    content_type='application/x-recordio')

s3_channels = {'train': train_data,
               'validation': validation_data}

In [66]:
ic.fit(inputs=s3_channels)

2022-09-11 00:38:53 Starting - Starting the training job...ProfilerReport-1662856732: InProgress
...
2022-09-11 00:39:36 Starting - Preparing the instances for training......
2022-09-11 00:40:37 Downloading - Downloading input data...
2022-09-11 00:41:17 Training - Downloading the training image.............Docker entrypoint called with argument(s): train
[09/11/2022 00:43:32 INFO 140408845727552] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/image_classification/default-input.json: {'use_pretrained_model': 0, 'num_layers': 152, 'epochs': 30, 'learning_rate': 0.1, 'lr_scheduler_factor': 0.1, 'optimizer': 'sgd', 'momentum': 0, 'weight_decay': 0.0001, 'beta_1': 0.9, 'beta_2': 0.999, 'eps': 1e-08, 'gamma': 0.9, 'mini_batch_size': 32, 'image_shape': '3,224,224', 'precision_dtype': 'float32'}
[09/11/2022 00:43:32 INFO 140408845727552] Merging with provided configuration from /opt/ml/input/config/hyperparameters.json: {'epochs': '10', 'mini_batch_size': '128', 'n

In [27]:
ic_predictor = ic.deploy(initial_instance_count=1,
                         instance_type='ml.t2.medium')

-----------------------!

In [28]:
import boto3, json
import numpy as np

In [38]:
with open('test.jpg', 'rb') as f:
    payload = f.read()
    payload = bytearray(payload)
client = boto3.client('sagemaker-runtime')
response = client.invoke_endpoint(
    EndpointName=ic_predictor.endpoint_name,
    Body=payload,
    ContentType='application/x-image',
)


In [39]:
result = response['Body'].read()
result

b'[0.006279913242906332, 0.9937200546264648]'

In [40]:
result = json.loads(result)

In [41]:
result

[0.006279913242906332, 0.9937200546264648]

In [42]:
index = np.argmax(result)

In [44]:
print(result[index], index)

0.9937200546264648 1


In [45]:
ic_predictor.delete_endpoint()